In [29]:
GYRO = False
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 20
MODEL_NAME = f"position_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}"
DEV_SIZE = 7
TEST_SIZE = 7

### This code trains a binary classifier which returns "moving" or "static" activity, currently trains on all data


In [30]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split
import split_by_student


STATIONARY = {
    "sitting",
    "standing",
    "lying_down_back",
    "lying_down_stomach",
    "lying_down_right",
    "lying_down_left"
}


MOVING = {
    "sitting",
    "standing",
    "lying_down_left",
    "lying_down_right",
    "lying_down_back",
    "lying_down_stomach",
    "walking",
    "ascending_stairs",
    "descending_stairs",
    "shuffle_walking",
    "running",
    "misc_movements"
}

OUTCOMES = ["stationary", "moving"]

LABEL_TO_INDEX = {label: idx for idx, label in enumerate(OUTCOMES)}


DATA_DIRECTORY = "./all_respeck"

In [31]:
# generate training data
def generate_training_data(directory, sequence_length, overlap, file_names, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in csv_dictionary.keys():

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if csv_file in file_names:
                if gyro:
                    sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
                else:
                    sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

                # iterate through each generated sequence
                for sequence in sequences:
                    if key.split("&")[0] in STATIONARY:
                        tagged_data.append(("stationary", sequence))
                    elif key.split("&")[0] in MOVING:
                        tagged_data.append(("moving", sequence))
                    else:
                        print("ERROR: activity not found")
                        return None

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [32]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    if len(validation_data[0]) == 0:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size)
    else:
        model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [33]:
def create_sequence_label_lists(tagged_sequences):
    sequences = [sequence for _, sequence in tagged_sequences]
    labels = [label for label, _ in tagged_sequences]
    sequences = np.array(sequences, dtype=np.float32)
    labels_encoded = [LABEL_TO_INDEX[label] for label in labels]
    labels = np.array(labels_encoded)

    return sequences, labels
    

def create_data_sets(dev_size, test_size):

    training_files, dev_files, test_files = split_by_student.split_data(students_in_dev_set= dev_size, students_in_test_set=test_size)

    tagged_training_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=training_files)
    tagged_dev_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=dev_files)
    tagged_test_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP, file_names=test_files)

    train_data, train_labels = create_sequence_label_lists(tagged_training_sequences)
    dev_data, dev_labels = create_sequence_label_lists(tagged_dev_sequences)
    test_data, test_labels = create_sequence_label_lists(tagged_test_sequences)

    #print(len(train_data), len(train_labels), len(dev_data), len(dev_labels), len(test_data), len(test_labels))

    return train_data, train_labels, dev_data, dev_labels, test_data, test_labels

In [34]:



def generate_binary_model():

    train_data, train_labels, dev_data, dev_labels, test_data, test_labels = create_data_sets(dev_size=DEV_SIZE, test_size=TEST_SIZE)

    # train and save model (CHOOSE BETWEEN CNN AND LSTM)
    model = train_model_CNN(train_data, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_data, test_labels)
    print (f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    return model

In [35]:
model = generate_binary_model()

# Save the trained model
#model.save("models/models_for_presentation/model1_for_presentation.keras")

Train Set: s98, s82, s59, s44, s70, s102, s18, s100, s40, s74, s48, s17, s38, s56, s84, s43, s36, s71, s45, s1, s13, s8, s42, s29, s86, s80, s11, s22, s93, s33, s88, s35, s61, s34, s79, s72, s9, s57, s96, s65, s52, s97, s21, s66, s32, s23, s55, s50, s75, s77, s60, s30, s83, s5, s7, s3, s54
Dev Set: s12, s95, s64, s92, s51, s16, s27
Test Set: s39, s67, s46, s63, s91, s87, s15
there are 67365 tagged sequences in the dataset
there are 8337 tagged sequences in the dataset
there are 7766 tagged sequences in the dataset
Epoch 1/20
6737/6737 [==============================] - 21s 3ms/step - loss: 0.0442 - accuracy: 0.9860 - val_loss: 0.0551 - val_accuracy: 0.9830
Epoch 2/20
6737/6737 [==============================] - 18s 3ms/step - loss: 0.0219 - accuracy: 0.9937 - val_loss: 0.0621 - val_accuracy: 0.9842
Epoch 3/20
6737/6737 [==============================] - 19s 3ms/step - loss: 0.0179 - accuracy: 0.9946 - val_loss: 0.0276 - val_accuracy: 0.9887
Epoch 4/20
6737/6737 [=======================